In [ ]:
import torch
from torch import nn
from torchvision import models
from torchvision import transforms

from tqdm import tqdm

import pandas as pd
from torch.utils.data import DataLoader, Dataset
import os

from PIL import Image

import numpy as np

In [ ]:
ipt = '/kaggle/input/planttraits2024'
opt = '/kaggle/working/'
netpth = '/kaggle/input/planttrails2024efficient/model.pth'
os.environ['CUDA_VISIBLE_DEVICES'] = "0, 1"

In [ ]:
class MultiModalNetwork(nn.Module):
    def __init__(self, num_classes=6):
        super(MultiModalNetwork, self).__init__()

        self.efficientnet_backbone = models.efficientnet_b0(pretrained=True)
        self.efficientnet_backbone = nn.Sequential(*list(self.efficientnet_backbone.children())[:-1])  # 最後の分類層を除外
        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.dropout1 = nn.Dropout(0.5)


        self.dense1 = nn.Linear(155, 326)
        self.dense2 = nn.Linear(326, 64)
        self.dropout2 = nn.Dropout(0.5)
        self.concat_dense = nn.Linear(1280 + 64, num_classes)

    def forward(self, image_input, numerical_input):
        if self.train:
            x1 = self.efficientnet_backbone(image_input)
            x1 = self.pooling(x1).view(-1, x1.shape[1])
            x1 = self.dropout1(x1)

            x2 = nn.functional.relu(self.dense1(numerical_input))
            x2 = nn.functional.relu(self.dense2(x2))
            x2 = self.dropout2(x2)


            x3 = torch.cat((x1, x2), dim=1)


            x3 = self.concat_dense(x3)
            return x3

        elif self.eval:

            x1 = self.efficientnet_backbone(image_input)
            x1 = self.pooling(x1).view(-1, x1.shape[1])

            x2 = nn.functional.relu(self.dense1(numerical_input))
            x2 = nn.functional.relu(self.dense2(x2))

            x3 = torch.cat((x1, x2), dim=1)

            x3 = self.concat_dense(x)

            return x3


class PlantTraitsDataset(Dataset):
    def __init__(self, csv_file, train=True, transform=None):
        self.transform = transform
        self.train = train

        df = pd.read_csv(csv_file)

        self.ids = df['id']

        self.features = df.iloc[:, 1:156]
        self.features = (self.features - self.features.mean(axis=0)) / self.features.std(axis=0)

        self.labels = df.iloc[:, 158:164]
        self.std = self.labels.std(axis=0)
        self.mean = self.labels.mean(axis=0)
        self.labels = (self.labels - self.mean) / self.std

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):

        if self.train:
            image_path = os.path.join(ipt, 'train_images', str(self.ids.iloc[idx]) + '.jpeg')
        else:
            image_path = os.path.join(ipt, 'test_images', str(self.ids.iloc[idx]) + '.jpeg')
        image = Image.open(image_path)
        if self.transform:
            image = self.transform(image)

        features = torch.tensor(self.features.iloc[idx].values.astype('float32'))
        if self.train:
            labels = torch.tensor(self.labels.iloc[idx].values.astype('float32'))
            return image, features, labels
        else:
            return image, features

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4482, 0.4525, 0.3360], std=[0.1086, 0.0971, 0.1172]),
])


test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4482, 0.4525, 0.3360], std=[0.1086, 0.0971, 0.1172]),
])


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:

torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = False

model = MultiModalNetwork(num_classes=6)
model = model.to(device)
model = torch.nn.DataParallel(model)

model.load_state_dict(torch.load(netpth))

In [ ]:
model.eval()
dataset = PlantTraitsDataset(os.path.join(ipt, 'train.csv'), transform=train_transform, train=True)
testset = PlantTraitsDataset(os.path.join(ipt, 'test.csv'), transform=test_transform, train=False)
test_loader = DataLoader(testset, batch_size=256, shuffle=True, num_workers=os.cpu_count(), pin_memory=True)

predictions = []
with torch.no_grad():
    with tqdm(total=len(test_loader)) as pbar:
        for i, (image, numerical) in enumerate(test_loader):
            image = image.to(device)
            numerical = numerical.to(device)
            output = model(image, numerical)
            pbar.update(1)
            predictions.append(output.detach().cpu().numpy())

In [ ]:
means = [0.3993, 18, 0.5, 1.7, 30, 1500]
stds = [0.3, 10, 1, 1, 50, 1500]

print(f'mean:{means} stds:{stds}')
predictions = np.concatenate(predictions, axis=0)
predictions = (predictions * stds ) + means

In [ ]:

ids = pd.read_csv(os.path.join(ipt, 'test.csv'))['id']


submission_df = pd.DataFrame(predictions, columns=['X4', 'X11', 'X18', 'X50', 'X26', 'X3112'])
submission_df.insert(0, 'id', ids)


submission_path = 'submission.csv'
submission_df.to_csv(submission_path, index=False)
submission_df

In [ ]:
test = pd.read_csv('/kaggle/input/planttraits2024/test.csv')
train =pd.read_csv('/kaggle/input/planttraits2024/train.csv')

train = train[train['X4_mean'] >= 0]
y_columns = [col for col in train.columns if col.endswith('_mean')]
target = train[y_columns]

target = target[['X4_mean', 'X11_mean', 'X18_mean', 'X50_mean', 'X26_mean', 'X3112_mean']]

In [ ]:
filtered_train = target[target['X4_mean'] >= 0]
filtered_train = filtered_train[filtered_train['X11_mean'] < 100]
filtered_train= filtered_train[filtered_train['X18_mean'] < 50]
filtered_train= filtered_train[filtered_train['X26_mean'] < 5000]
filtered_train= filtered_train[filtered_train['X50_mean'] < 10]
filtered_train[filtered_train['X3112_mean'] < 25000]

mean_values = filtered_train.mean()
prediction_columns = ['X4', 'X11', 'X18', 'X50', 'X26', 'X3112']

submission = pd.DataFrame({'id': test['id']})
submission[prediction_columns] = mean_values
submission

In [ ]:
submission_df = submission[prediction_columns] + submission_df[prediction_columns]
submission_df.insert(0, 'id', ids)

submission_df

In [ ]:
submission_df.to_csv(submission_path, index=False)